In [ ]:
import tensorflow as tf
# Get the GPU device name.
device_name = tf.test.gpu_device_name()
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving bert_davidson.pt to bert_davidson.pt


In [ ]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    print(torch.cuda.get_device_name())
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80
Tesla K80


In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
# Load the dataset into a pandas dataframe.
df = pd.read_csv("./reddit_comments_race_black_processed.csv", encoding='latin-1')
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
# Display 10 random rows from the data.
df = df.dropna()
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

df.sample(10)

Number of training sentences: 8,469

Number of training sentences: 8,453



,id,comments,comments_processed
5473,fns3f32,&gt; so I guess blacks are just more violent a...,gt so i guess blacks are just more violent and...
6383,dl2k3ey,It's because there are far more whites overall...,its because there are far more whites overall....
7477,eldxyw4,Not all blacks are like that! But what can th...,not all blacks are like that! but what can the...
4701,e5urphu,"Saying someone is ""pretty for a black girl"" is...",saying someone is pretty for a black girl is i...
8031,fbdj8xe,You're saying that every single black person i...,youre saying that every single black person is...
7411,f7k4o3w,"""100% of Blacks are gang banging criminals""\n\...",of blacks are gang banging criminals blanket s...
175,euwk9sn,&gt; Those pesky fbi stats says say blacks mal...,gt those pesky fbi stats says say blacks males...
2639,f06dvfb,"Yeah man, calling somebody who is implying tha...",yeah man calling somebody who is implying that...
141,f4b83eb,No. Most people who repeat things like this us...,no. most people who repeat things like this us...
7244,fsegk3l,"Hmm, maybe black people shouldn't have to eith...",hmm maybe black people shouldnt have to either...


In [ ]:
sentences = df.comments_processed.values
# df['Code'] = df['Code'].astype('category').cat.codes
# labels = df.code_label.values
# labels = df['class'].values
print(sentences.dtype)
# print(labels.dtype)
print(sentences[:10])
# print(labels[:10])

object
['tell me sir why do you think afro-americans are prone to commiting more violent crimes?'
 'when europeans colonize stuff and act extra violently we are risk takers and rebels when africans are violent poor impulse control'
 'maybe that has to do with the fact that african americans are statistically more likely to commit violent crimes?'
 'so what does this video try to prove that mexicans and african americans are violent troublemakers or something?'
 'african americans are not more violent than whites when you adjust for income. the real emergency is african american poverty.'
 'the statistic is about violent crimes or murders both of which black people are responsible for about of.'
 'gt how potentially millions of innocent african americans are victims of violent crimes by the police each yea you did no such thing'
 'yep its totally because of melanine! that shit causes so much violence! because black people are inherently more violent and criminal right?'
 'the violent cr

In [ ]:
from transformers import BertTokenizer
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
input_ids = []
# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    if len(sent.split()) > 200:
      # sent = sent[:512]
      print('in')
      sent = ' '.join(sent.split()[:100])
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True)
    #print(tokenizer.tokenize(sent))
    
    # Add the encoded sentence to the list.
    #print(encoded_sent)
    input_ids.append(encoded_sent)
# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])
#print(tokenizer.decode(input_ids=[12731, 3372]))

Original:  tell me sir why do you think afro-americans are prone to commiting more violent crimes?
Token IDs: [101, 2425, 2033, 2909, 2339, 2079, 2017, 2228, 17694, 1011, 4841, 2024, 13047, 2000, 10797, 2075, 2062, 6355, 6997, 1029, 102]


In [ ]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))
print(len(input_ids[0]))
long_sen = 0
for sen in input_ids:
  if len(sen) > 64:
    long_sen += 1
print(long_sen)

Max sentence length:  44
21
0


In [ ]:
from keras.preprocessing.sequence import pad_sequences
# Set the maximum sequence length.

MAX_LEN = 64
print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))
# Pad our input tokens with value 0.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
print('\Done.')


Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0
\Done.


In [ ]:
# Create attention masks
attention_masks = []
# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [ ]:
test_inputs = torch.tensor(input_ids)
test_masks = torch.tensor(attention_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32
test_data = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
model = torch.load('./bert_davidson.pt')
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
import time
import datetime
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# ========================================
#               Test
# ========================================
# After the completion of each training epoch, measure our performance on
# our validation set.
print("")
print("Running Validation...")
t0 = time.time()
# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()
# Tracking variables 

# Evaluate data for one epoch
pred_labels_test = []

for batch in test_dataloader:
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch
    
    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        
        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # Get the "logits" output by the model. The "logits" are the output
    # values prior to applying an activation function like the softmax.
    logits = outputs[0]
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    pred_flat = np.argmax(logits, axis=1).flatten()
    
    # Calculate the accuracy for this batch of test sentences.
    pred_labels_test.append(pred_flat)


Running Validation...


In [ ]:
print(len(pred_labels_test))
print(pred_labels_test)

265
[array([0, 2, 0, 0, 0, 2, 2, 1, 2, 2, 2, 1, 0, 0, 0, 2, 2, 1, 0, 0, 1, 2,
       1, 0, 0, 2, 2, 0, 0, 1, 2, 2]), array([1, 2, 0, 0, 1, 0, 2, 1, 0, 1, 2, 0, 2, 1, 2, 2, 0, 0, 0, 0, 0, 2,
       2, 2, 1, 1, 2, 2, 0, 0, 1, 0]), array([2, 1, 0, 1, 2, 1, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2,
       0, 2, 0, 0, 2, 1, 0, 1, 1, 2]), array([1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 2, 2, 0, 0, 1, 0, 1, 1, 2, 1,
       2, 0, 0, 1, 2, 0, 1, 2, 1, 2]), array([0, 1, 0, 1, 2, 0, 1, 2, 2, 2, 2, 2, 1, 0, 0, 0, 0, 0, 2, 1, 1, 1,
       1, 0, 0, 1, 2, 0, 0, 0, 0, 1]), array([0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 0,
       2, 2, 2, 0, 0, 0, 1, 2, 2, 0]), array([0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 1, 0, 1, 2, 1, 0, 2, 2, 2, 2, 1, 1,
       2, 2, 2, 1, 0, 2, 1, 1, 2, 2]), array([0, 0, 0, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 0, 0, 2, 0, 0, 1, 0, 2,
       0, 0, 1, 1, 0, 2, 0, 1, 2, 2]), array([0, 0, 0, 0, 1, 2, 1, 0, 2, 0, 2, 2, 1, 0, 2, 0, 0, 1, 0, 0, 0, 1,
       2, 1, 0, 1, 2, 0, 2

In [ ]:
preds = []
for l in pred_labels_test:
  l = l.tolist()
  preds.append(l)

preds_flat = [item for sublist in preds for item in sublist]
print(preds_flat)
print(len(preds_flat))


[0, 2, 0, 0, 0, 2, 2, 1, 2, 2, 2, 1, 0, 0, 0, 2, 2, 1, 0, 0, 1, 2, 1, 0, 0, 2, 2, 0, 0, 1, 2, 2, 1, 2, 0, 0, 1, 0, 2, 1, 0, 1, 2, 0, 2, 1, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 2, 2, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2, 1, 0, 1, 1, 2, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 2, 2, 0, 0, 1, 0, 1, 1, 2, 1, 2, 0, 0, 1, 2, 0, 1, 2, 1, 2, 0, 1, 0, 1, 2, 0, 1, 2, 2, 2, 2, 2, 1, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 0, 2, 2, 2, 0, 0, 0, 1, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 1, 0, 1, 2, 1, 0, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 0, 2, 1, 1, 2, 2, 0, 0, 0, 1, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 0, 0, 2, 0, 0, 1, 0, 2, 0, 0, 1, 1, 0, 2, 0, 1, 2, 2, 0, 0, 0, 0, 1, 2, 1, 0, 2, 0, 2, 2, 1, 0, 2, 0, 0, 1, 0, 0, 0, 1, 2, 1, 0, 1, 2, 0, 2, 2, 2, 2, 1, 0, 0, 0, 0, 1, 2, 0, 0, 2, 0, 1, 2, 2, 2, 2, 0, 2, 1, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 1, 1, 